In [112]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [113]:
train_data = pd.read_csv("train.csv")
train_data.isnull().any()


PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [114]:
train_data['Age'].fillna((train_data['Age'].mean()),inplace=True)
train_data=train_data.drop(['Embarked','Cabin','Fare','Ticket','Name'],axis=1)

In [115]:
test_data = pd.read_csv("test.csv")
test_data['Age'].fillna((test_data['Age'].mean()),inplace=True)
test_data=test_data.drop(['Embarked','Cabin','Fare','Ticket','Name'],axis=1)


In [116]:
#the below code is used for debug purpose
#Here I am checking for columns with NA
train_data.columns[train_data.isna().any()].tolist()

[]

In [117]:
train_data["FamilyGroup"] = train_data["SibSp"] + train_data["Parch"]
test_data["FamilyGroup"] = test_data["SibSp"] + test_data["Parch"]
train_data["AgeGroup"] = pd.cut(train_data["Age"], bins=[0, 18, 35, 50, 100], labels=["Child", "Adult1", "Adult2", "Elderly"])
test_data["AgeGroup"] = pd.cut(test_data["Age"], bins=[0, 18, 35, 50, 100], labels=["Child", "Adult1", "Adult2", "Elderly"])
features = ["Pclass", "Sex", "FamilyGroup", "AgeGroup"]

In [118]:
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]
X_test = pd.get_dummies(test_data[features])

In [119]:
# Tune Model Hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
"n_estimators": [50, 100, 150],
"max_depth": [3, 5, 7],
"min_samples_split": [2, 5, 10]
}
rf = RandomForestClassifier(n_estimators=500, max_depth=5,random_state=1)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search.fit(X, y)

best_rf = grid_search.best_estimator_

In [120]:
X = X.fillna(X.mean())
X_test = X_test.fillna(X_test.mean())

In [125]:
# Perform Cross-validation

from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5)
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

Cross-validation scores: [0.84916201 0.83146067 0.8258427  0.80337079 0.83707865]
Mean cross-validation score: 0.8293829640323898


In [102]:
best_rf.fit(X, y)
predictions = best_rf.predict(X_test)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
